In [1]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np
import yaml

In [2]:
train_size = os.listdir("Dataset/train/images")
print(len(train_size))

21203


In [3]:
from ultralytics import YOLO

training_model = YOLO("yolov8n.yaml")


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

In [4]:
import torch
torch.cuda.is_available()

True

In [24]:
train_images = os.listdir("Dataset/train/images")
train_labels = os.listdir("Dataset/train/labels")
REDUCED_SIZE = 8000

print(len(os.listdir("Dataset/train/images")))
for image, label in zip(train_images[REDUCED_SIZE:], train_labels[REDUCED_SIZE:]):
    os.remove(os.path.join("Dataset/train/labels", label))
    os.remove(os.path.join("Dataset/train/images", image))
# print(os.listdir("Dataset/train/labels"))
print(len(os.listdir("Dataset/train/labels")))

21201
8000


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'Dataset/train/images'

In [26]:
results = training_model.train(data="Dataset/data.yaml", epochs=5, device=0)

New https://pypi.org/project/ultralytics/8.0.171 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.159  Python-3.11.0 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.yaml, data=Dataset/data.yaml, epochs=5, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms

In [32]:
results_new = training_model.predict("Dataset/train/images/000056694_jpg.rf.d3044032743b059a02077deb4bbf4cea.jpg")


image 1/1 C:\Users\Aman's Laptop\Documents\DNNProjects\PokerHandDetection\Dataset\train\images\000056694_jpg.rf.d3044032743b059a02077deb4bbf4cea.jpg: 640x640 (no detections), 31.7ms
Speed: 0.0ms preprocess, 31.7ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)


In [5]:
from ultralytics import YOLO
model = YOLO("runs/detect/train4/weights/best.pt")

In [9]:
import cv2

image = cv2.imread("Dataset/train/images/000056694_jpg.rf.d3044032743b059a02077deb4bbf4cea.jpg")
result = model.predict(image)

if image is not None:
    # Display the image in a window
    result_arr = result[0].boxes.data
    item_dict = result[0].names
    print(len(result_arr))
    for result in result_arr:
        cv2.rectangle(image, (int(result[0]), int(result[1])), (int(result[2]), int(result[3])), (120, int(result[5])/80 * 255, (1-int(result[5]))/80 * 255), 5)
    cv2.imshow('Image', image)

    # Wait for a key press and close the window
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print("Image not found or couldn't be loaded.")


0: 640x640 (no detections), 33.4ms
Speed: 7.7ms preprocess, 33.4ms inference, 3.5ms postprocess per image at shape (1, 3, 640, 640)


0
